In [1]:
""" 
Author: maxwell.cook@colorado.edu
"""

import os, sys, time
import pandas as pd
import rioxarray as rxr
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import to_rgba

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [2]:
# Load the VIIRS active fire detections gridded data
fp = os.path.join(projdir,'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_fires_pixar_gridstats.gpkg')
grid = gpd.read_file(fp)
grid.columns

Index(['grid_index', 'grid_area', 'afd_count', 'unique_days', 'overlap',
       'frp_csum', 'frp_max', 'frp_min', 'frp_mean', 'frp_p90', 'frp_p95',
       'frp_p97', 'frp_p99', 'frp_first', 'day_max_frp', 'dt_max_frp',
       'first_obs_date', 'last_obs_date', 't4_max', 't4_mean', 't5_max',
       't5_mean', 'day_count', 'night_count', 'frp_max_day', 'frp_max_night',
       'frp_csum_day', 'frp_csum_night', 'frp_mean_day', 'frp_mean_night',
       'frp_p90_day', 'frp_p90_night', 'frp_p95_day', 'frp_p95_night',
       'frp_p97_day', 'frp_p97_night', 'frp_p99_day', 'frp_p99_night',
       'frp_first_day', 'frp_first_night', 'Fire_ID', 'Fire_Name', 'geometry'],
      dtype='object')

In [ ]:
print(f"There are [{len(grid['Fire_ID'].unique())}] fire events in the Southern Rockies (2018-2023)")
print(f"\tWith {len(grid)} gridcells.")

In [3]:
# load and prepare our study region for cropping TreeMap
# Southern Rockies ecoregion bounds (buffered)
fp = os.path.join(projdir,'data/spatial/raw/boundaries/na_cec_eco_l3_srme.gpkg')
srm = gpd.read_file(fp)
# Crop the raster by the SRM bounds
srm['geometry'] = srm.geometry.buffer(10000)
bounds = srm.total_bounds # total bounds of ecoregion
bounds

array([-1193290.29502988,  1391628.00599962,  -683136.18714099,
        2253336.36986925])

### USFS ca. 2016 TreeMap (Algorithm Forest Type Code)


In [ ]:
# Load the summary table (from GEE reductions)
fp = os.path.join(projdir,'earth-engine/exports/treemap/gridstats_fortypcd.csv')
fortypcd = pd.read_csv(fp)
fortypcd.drop(columns=['system:index','.geo'], inplace=True) # drop the GEE id and .geo columns
fortypcd.head()

In [ ]:
print(fortypcd['species_histogram'].head())
print(fortypcd['species_histogram'].map(type).value_counts())

In [ ]:
# unpack histogram dictionary into columns
def parse_histogram(hist_str):
    # Convert float or NaN to string, then parse
    if isinstance(hist_str, float) or pd.isna(hist_str):
        hist_str = str(int(hist_str)) if not pd.isna(hist_str) else ""
    kv_pairs = hist_str.split(', ')  # Split the key-value pairs
    # Convert key-value pairs to a dictionary
    return {int(kv.split(':')[0]): float(kv.split(':')[1]) for kv in kv_pairs if ':' in kv}

# parse the dictionary
fortypcd['species_histogram'] = fortypcd['species_histogram'].apply(parse_histogram)

hist_df = fortypcd['species_histogram'].apply(pd.Series)
fortypcd = pd.concat([fortypcd.drop(columns=['species_histogram']), hist_df], axis=1).reset_index(drop=True)
print("Adjusted DataFrame:")
fortypcd.head()

In [ ]:
# Pivot longer and join to get species name
fortypcd_l = fortypcd.melt(id_vars=['grid_index','total_pixels'], var_name='FORTYPCD', value_name='count')
fortypcd_l['FORTYPCD'] = fortypcd_l['FORTYPCD'].astype(int)

# Drop 'NaN' counts (species not present in the AFD)
fortypcd_l.dropna(subset=['count'], inplace=True)
fortypcd_l.sort_values(by=['grid_index'], inplace=True)
print("Dropped NA values.")
print(fortypcd_l.columns)
fortypcd_l.head(3)

In [ ]:
# load the species lookup table
fp = os.path.join(projdir,'tabular/mod/treemap_fortypcd_species_mapping.csv')
species_df = pd.read_csv(fp)
species_df = species_df[['FORTYPCD','SpeciesName']]
species_df = species_df[species_df['FORTYPCD'].isin(fortypcd_l['FORTYPCD'].unique())]
print(species_df.columns)

# merge with grid summary to get forest species name
fortypcd_l_ = fortypcd_l.merge(species_df, on='FORTYPCD', how='left')
fortypcd_l_.head(3)

In [ ]:
# print a unqiue list of tree species
fortypcd_l_['SpeciesName'].unique()

In [ ]:
# calculate the total forest count and percent cover for each species
forest_count = fortypcd_l_.groupby('grid_index')['count'].sum().reset_index()
forest_count = forest_count.rename(columns={'count': 'forest_pix'})

# Join back to the data frame, calculate percent cover
fortypcd_l_pct = fortypcd_l_.merge(forest_count, on='grid_index', how='left')

fortypcd_l_pct['spp_pct'] = (fortypcd_l_pct['count'] / fortypcd_l_pct['forest_pix']) * 100
fortypcd_l_pct['forest_pct'] = (fortypcd_l_pct['forest_pix'] / fortypcd_l_pct['total_pixels']) * 100

# reshape the columns
cols = ['grid_index', 'FORTYPCD', 'SpeciesName', 'count', 'forest_pix', 'total_pixels', 'spp_pct', 'forest_pct']
fortypcd_l_pct = fortypcd_l_pct[cols]

# check the results
fortypcd_l_pct = fortypcd_l_pct.sort_values(by=['grid_index'])
print(fortypcd_l_pct.columns)
fortypcd_l_pct.head(3)

In [ ]:
# Create a histogram of the forest species (prevalence)

In [ ]:
fortyp_df = fortypcd_l_pct.copy() # make a copy to work with
fortyp_df = fortyp_df[fortyp_df['count'] > 0] # make sure there are some pixels

# calculate the landscape fraction (prevalence)
spp_prevalence = fortyp_df.groupby('SpeciesName')['count'].sum().reset_index()
spp_prevalence['fraction'] = spp_prevalence['count'] / spp_prevalence['count'].sum()
spp_prevalence = spp_prevalence.sort_values(by='fraction', ascending=False)

# which species make up 97% of observations?
spp_prevalence['c_fraction'] = spp_prevalence['fraction'].cumsum()
top_species = spp_prevalence[spp_prevalence['c_fraction'] <= 0.97]
print(f"\n\nSpecies making up 97% of the total: \n\n{top_species}\n\n")

# plot it
plt.figure(figsize=(6, 4))
sns.barplot(data=top_species, x='SpeciesName', y='fraction', palette='viridis')
plt.xlabel('Species')
plt.ylabel('Fractional cover')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# perform a remap for some forest types
# create a 'Spruce-fir' category
# 'Pinon-juniper woodland' category

In [ ]:
spp_remap = {
    'Engelmann spruce': 'Spruce-fir',
    'Subalpine fir': 'Spruce-fir',
    'White fir': 'Spruce-fir',
    'Grand fir': 'Spruce-fir',
    'Engelmann spruce / subalpine fir': 'Spruce-fir',
    'Western juniper': 'Pinon-juniper woodland',
    'Rocky Mountain juniper': 'Pinon-juniper woodland',
    'Pinyon / juniper woodland': 'Pinon-juniper woodland',
    'Juniper woodland': 'Pinon-juniper woodland'
}

# Add a new column for the merged categories
fortyp_df['SpeciesName'] = fortyp_df['SpeciesName'].map(spp_remap).fillna(fortyp_df['SpeciesName'])
fortyp_df['SpeciesName'].unique()

In [ ]:
# re-plot the dataframe
# Calculate the total percent cover (prevalence)
spp_prevalence = fortyp_df.groupby('SpeciesName')['count'].sum().reset_index()
spp_prevalence['fraction'] = spp_prevalence['count'] / spp_prevalence['count'].sum()
spp_prevalence = spp_prevalence.sort_values(by='fraction', ascending=False)

# which species make up 97% of observations?
spp_prevalence['c_fraction'] = spp_prevalence['fraction'].cumsum()
top_species = spp_prevalence[spp_prevalence['c_fraction'] <= 0.97]
print(f"\n\nSpecies making up 97% of the total: \n\n{top_species}\n\n")

# plot it
plt.figure(figsize=(6, 4))
sns.barplot(data=top_species, x='SpeciesName', y='fraction', palette='viridis')
plt.title('Burned area forest type fraction')
plt.xlabel('Species')
plt.ylabel('Fractional cover')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# join the forest type dataframe to our FRP gridcells

In [ ]:
grid_ = grid.merge(fortyp_df, on='grid_index', how='left')
grid_.columns

In [ ]:
# subset the grids to predominantly forested plots (>50% forested)
grid_forest = grid_[grid_['forest_pct'] > 50]
print(f"{len(grid_forest['grid_index'].unique())}/{len(grid)} [{round(len(grid_forest['grid_index'].unique())/len(grid)*100,1)}%] gridcells are forested.")

In [ ]:
print(top_species['SpeciesName'].unique()) # check on our top species
# subset columns in our forested gridcells to retain just our top species
grid_forest = grid_forest[grid_forest['SpeciesName'].isin(top_species['SpeciesName'].unique())]
len(grid_forest['grid_index'].unique())

In [ ]:
# Load the treemap metrics, apply the same remapping

In [ ]:
ftyp_met = os.path.join(projdir,'earth-engine/exports/treemap/gridstats_fortypcd_metrics.csv')
tm_stats = pd.read_csv(ftyp_met)
tm_stats.drop(columns=['system:index','.geo'], inplace=True)
tm_stats.columns

In [ ]:
# unpack histogram dictionary into columns
def parse_histogram(hist_str):
    # Convert float or NaN to string, then parse
    if isinstance(hist_str, float) or pd.isna(hist_str):
        hist_str = str(int(hist_str)) if not pd.isna(hist_str) else ""
    kv_pairs = hist_str.split(', ')  # Split the key-value pairs
    # Convert key-value pairs to a dictionary
    return {kv.split(':')[0]: float(kv.split(':')[1]) for kv in kv_pairs if ':' in kv}

# parse the dictionary
tm_stats['species_metrics'] = tm_stats['species_metrics'].apply(parse_histogram)
hist_df = tm_stats['species_metrics'].apply(pd.Series)
tm_stats = pd.concat([tm_stats.drop(columns=['species_metrics']), hist_df], axis=1).reset_index(drop=True)
print("Adjusted DataFrame:")
tm_stats.head(3)

In [ ]:
# isolate melt bands/columns
band_names = ['BALIVE', 'SDIPCT_RMRS', 'STANDHT', 'TPA_DEAD', 'TPA_LIVE']
regex_p = '|'.join(band_names) # regex pattern for melting columns
metrics_cols = [col for col in tm_stats.columns if '_mn' in col]
print(f"[{len(metrics_cols)}] total metrics columns selected for melting:\n")

# do the melt
tm_stats_l = tm_stats.melt(
    id_vars=['grid_index'], 
    value_vars=metrics_cols, 
    var_name='metric', 
    value_name='value'
)
tm_stats_l.head(3)

In [ ]:
# gather the forest type code from the band name
tm_stats_l['FORTYPCD'] = tm_stats_l['metric'].str.extract(r'^(\d+)_')[0].astype(int)
tm_stats_l['metric'] = tm_stats_l['metric'].str.extract(f'({regex_p})')[0]
tm_stats_l = tm_stats_l[['grid_index','FORTYPCD','metric','value']]
tm_stats_l.dropna(subset=['value'], inplace=True) # drop NaNs
tm_stats_l['metric'].unique()

In [ ]:
# pivot longer to get metrics as columns
tm_stats_lp = tm_stats_l.pivot_table(index=['grid_index', 'FORTYPCD'], columns='metric', values='value')
tm_stats_lp = tm_stats_lp.reset_index() # collapse the index
tm_stats_lp = tm_stats_lp[['grid_index','FORTYPCD']+band_names]
tm_stats_lp.rename(columns={'SDIPCT_RMRS': 'SDIPCT'}, inplace=True)
tm_stats_lp.head(3)

In [ ]:
# load the Sentinel-2 metrics

In [ ]:
fp = os.path.join(projdir,'earth-engine/exports/treemap/gridstats_fortypcd_s2.csv')
s2_stats = pd.read_csv(fp)
s2_stats.drop(columns=['system:index','.geo'], inplace=True)
s2_stats.columns

In [ ]:
# unpack histogram dictionary into columns
def parse_histogram(hist_str):
    # Convert float or NaN to string, then parse
    if isinstance(hist_str, float) or pd.isna(hist_str):
        hist_str = str(int(hist_str)) if not pd.isna(hist_str) else ""
    kv_pairs = hist_str.split(', ')  # Split the key-value pairs
    # Convert key-value pairs to a dictionary
    return {kv.split(':')[0]: float(kv.split(':')[1]) for kv in kv_pairs if ':' in kv}
# parse the dictionary
s2_stats['species_s2'] = s2_stats['species_s2'].apply(parse_histogram)
s2_stats.head(3)

In [ ]:
hist_df = s2_stats['species_s2'].apply(pd.Series)
s2_stats = pd.concat([s2_stats.drop(columns=['species_s2']), hist_df], axis=1).reset_index(drop=True)
print("Adjusted DataFrame:")
s2_stats.head(3)

In [ ]:
# extract the correct band names / columns
band_names = ['LAI', 'MNDWI']
regex_p = '|'.join(band_names) # regex pattern for melting columns
metrics_cols = [col for col in s2_stats.columns if 'LAI' or 'MNDWI' in col]
print(f"[{len(metrics_cols)}] total metrics columns selected for melting:\n")
# do the melt
s2_stats_l = s2_stats.melt(
    id_vars=['grid_index'], 
    value_vars=metrics_cols, 
    var_name='metric', 
    value_name='value'
)
s2_stats_l.head(3)

In [ ]:
# gather the forest type code
s2_stats_l['FORTYPCD'] = s2_stats_l['metric'].str.extract(r'^(\d+)_')[0].astype(int)
s2_stats_l['metric'] = s2_stats_l['metric'].str.extract(f'({regex_p})')[0]
s2_stats_l = s2_stats_l[['grid_index','FORTYPCD','metric','value']]
s2_stats_l.dropna(subset=['value'], inplace=True) # drop NaNs
s2_stats_l['metric'].unique()

In [ ]:
# pivot longer the get metrics as columns
s2_stats_lp = s2_stats_l.pivot_table(index=['grid_index', 'FORTYPCD'], columns='metric', values='value')
s2_stats_lp = s2_stats_lp.reset_index() # collapse the index
s2_stats_lp = s2_stats_lp[['grid_index','FORTYPCD','LAI','MNDWI']]
s2_stats_lp.head(3)

In [ ]:
# Merge the TreeMap metrics with the Sentinel-2 metrics
# Merge to the FRP grid

In [ ]:
# Join to the FORTYPCD dataframe by grid index and FORTYPCD
treemap_df = fortyp_df.merge(tm_stats_lp, on=['grid_index','FORTYPCD'], how='left')
treemap_df = treemap_df.merge(s2_stats_lp, on=['grid_index','FORTYPCD'], how='left')

# Make sure we're just keeping our 97% landscape fraction forest types
treemap_df = treemap_df[treemap_df['SpeciesName'].isin(top_species['SpeciesName'].unique())]

print(treemap_df.columns)
treemap_df['SpeciesName'].unique()

In [ ]:
treemap_df.head(3)

In [ ]:
# save the TreeMap table out.
out_fp = os.path.join(projdir,'tabular/mod/viirs_snpp_jpss1_gridstats_treemap-s2.csv')
treemap_df.to_csv(out_fp)
print(f"Saved file to: {out_fp}")

In [ ]:
# Identify the dominant forest type for each gridcell
# Subset to forested gridcells (>50%)

In [ ]:
dtree = treemap_df.loc[treemap_df.groupby('grid_index')['spp_pct'].idxmax()]
dtree = dtree[dtree['forest_pct'] > 50] # greater than 50% forested
print(dtree['SpeciesName'].unique())

In [ ]:
# merge to the FRP gridcells
grid_dtree = grid.merge(dtree, on='grid_index', how='inner')
print(grid_dtree.columns)

In [ ]:
# plot the distribution of FRP max and cumulative
print(f"Maximum FRP distribution:\n {grid_dtree['frp_max'].describe()}\n")
print(f"Cumulative FRP distribution:\n {grid_dtree['frp_csum'].describe()}\n")

In [ ]:
# drop any 0 FRP gridcells
print(f"Dropping [{len(grid_dtree[grid_dtree['frp_max'] == 0])}] gridcells w/ 0 FRP.")
grid_dtree = grid_dtree[grid_dtree['frp_max'] > 0]

In [ ]:
# Plot FRP by EVT group name
# Find the median and sort the evt classes
med = grid_dtree.groupby('SpeciesName')['frp_csum'].median().sort_values(ascending=False)
sorted = med.index.tolist()

grid_dtree['frp_max_adj'] = grid_dtree['frp_max'] + 1
grid_dtree['frp_csum_adj'] = grid_dtree['frp_csum'] + 1

# create a dataframe with dominance (>90% of the same species)
dom = grid_dtree[grid_dtree['spp_pct'] > 75]

# plot it
plt.figure(figsize=(8, 6)) 

sns.boxplot(
    data=grid_dtree, 
    x='frp_csum_adj', 
    y='SpeciesName', 
    order=sorted, 
    palette='coolwarm_r', 
    dodge=False)

plt.xscale('log')

plt.xlabel('Fire Radiative Power (FRP)')
plt.ylabel('Forest Type')

plt.tight_layout()
plt.show()

In [ ]:
# plot total detections by forest type

In [ ]:
total_afd = grid_dtree['afd_count'].sum() # total afd counts
# proportional counts
afd_props = (
    grid_dtree.groupby('SpeciesName')['afd_count']
    .sum()
    .reset_index(name='afd_count')
)
afd_props['afd_prop'] = afd_props['afd_count'] / total_afd
# merge with the species prevalence
afd_props_spp = pd.merge(afd_props, spp_prevalence, on='SpeciesName', how='inner')
afd_props_spp.head(7)

In [ ]:
# plot the difference between landscape fraction and afd
plt.figure(figsize=(8, 5))
sns.barplot(
    data=afd_props_spp.melt(
        id_vars='SpeciesName',
        value_vars=['afd_prop', 'fraction'],
        var_name='Proportion Type',
        value_name='Proportion'
    ),
    x='SpeciesName',
    y='Proportion',
    hue='Proportion Type',
    palette='viridis'
)

plt.title('Proportion of Fire Detections vs. Landscape Proportion by Species')
plt.xlabel('Dominant Tree Species')
plt.ylabel('Proportion')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Proportion Type')
plt.tight_layout()
plt.show()

## LANDFIRE Existing Vegetation Type (EVT) c.a. 2016

In [4]:
fp = os.path.join(maindir,'data/landcover/LANDFIRE/LF2016_EVT_200_CONUS/Tif/LC16_EVT_200.tif')
evt_da = rxr.open_rasterio(fp, masked=True, cache=False, chunks='auto').squeeze()
shp, gt, wkt, nd = evt_da.shape, evt_da.spatial_ref.GeoTransform, evt_da.rio.crs, evt_da.rio.nodata
print(
    f"Shape: {shp}; \n"
    f"GeoTransform: {gt}; \n"
    f"WKT: {wkt}; \n"
    f"NoData Value: {nd}; \n"
    f"Data Type: {evt_da[0].dtype}")
gc.collect()

Shape: (97283, 154207); 
GeoTransform: -2362425.0 30.0 0.0 3177435.0 0.0 -30.0; 
WKT: EPSG:5070; 
NoData Value: nan; 
Data Type: float32


54

In [5]:
evt_da_crop = evt_da.rio.clip_box(
    minx=bounds[0]+1000, # +1000 meters for a small buffer
    miny=bounds[1]+1000, 
    maxx=bounds[2]+1000, 
    maxy=bounds[3]+1000
)
del evt_da
gc.collect()

47

In [6]:
t0 = time.time()

# Get the proportional landcover
grid_evt = compute_band_stats(
    geoms=grid[['grid_index','geometry']], 
    image_da=evt_da_crop, 
    id_col='grid_index'
)

t1 = (time.time() - t0) / 60
print(f"Total elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

Total elapsed time: 0.60 minutes.

~~~~~~~~~~



In [7]:
grid_evt.head()

,grid_index,evt,count,total_pixels,pct_cover
0,1955419,7051,21,169,12.426036
1,1955419,7052,146,169,86.390533
2,1955419,7054,2,169,1.183432
3,1955420,7051,37,182,20.329670
4,1955420,7052,145,182,79.670330


In [8]:
# Read in the lookup table for the EVT codes
lookup = os.path.join(maindir,'data/landcover/LANDFIRE/LF2016_EVT_200_CONUS/CSV_Data/LF16_EVT_200.csv')
lookup = pd.read_csv(lookup)
print(lookup.columns)

Index(['VALUE', 'EVT_NAME', 'LFRDB', 'EVT_FUEL', 'EVT_FUEL_N', 'EVT_LF',
       'EVT_PHYS', 'EVT_GP', 'EVT_GP_N', 'SAF_SRM', 'EVT_ORDER', 'EVT_CLASS',
       'EVT_SBCLS', 'R', 'G', 'B', 'RED', 'GREEN', 'BLUE'],
      dtype='object')


In [9]:
# Subset the codes we want to join, join back to the dataframe
lookup = lookup[['VALUE','EVT_NAME','EVT_PHYS','EVT_GP_N','EVT_CLASS','EVT_SBCLS','SAF_SRM']]
grid_evt_ = grid_evt.merge(lookup, left_on='evt', right_on='VALUE', how='left')
grid_evt_.drop('VALUE', axis=1, inplace=True)
grid_evt_.head()

,grid_index,evt,count,total_pixels,pct_cover,EVT_NAME,EVT_PHYS,EVT_GP_N,EVT_CLASS,EVT_SBCLS,SAF_SRM
0,1955419,7051,21,169,12.426036,Southern Rocky Mountain Dry-Mesic Montane Mixe...,Conifer,Douglas-fir-Ponderosa Pine-Lodgepole Pine Fore...,Closed tree canopy,Evergreen closed tree canopy,SAF 210: Interior Douglas-Fir
1,1955419,7052,146,169,86.390533,Southern Rocky Mountain Mesic Montane Mixed Co...,Conifer,Douglas-fir-Grand Fir-White Fir Forest and Woo...,Closed tree canopy,Evergreen closed tree canopy,SAF 211: White Fir
2,1955419,7054,2,169,1.183432,Southern Rocky Mountain Ponderosa Pine Woodland,Conifer,"Ponderosa Pine Forest, Woodland and Savanna",Open tree canopy,Evergreen open tree canopy,SAF 237: Interior Ponderosa Pine
3,1955420,7051,37,182,20.329670,Southern Rocky Mountain Dry-Mesic Montane Mixe...,Conifer,Douglas-fir-Ponderosa Pine-Lodgepole Pine Fore...,Closed tree canopy,Evergreen closed tree canopy,SAF 210: Interior Douglas-Fir
4,1955420,7052,145,182,79.670330,Southern Rocky Mountain Mesic Montane Mixed Co...,Conifer,Douglas-fir-Grand Fir-White Fir Forest and Woo...,Closed tree canopy,Evergreen closed tree canopy,SAF 211: White Fir


In [ ]:
evt_counts = grid_evt_.groupby(['evt','EVT_NAME','EVT_GP_N','SAF_SRM']).size().reset_index(name='counts')
evt_counts.sort_values('counts', ascending=False, inplace=True)
evt_counts.head(10)

In [ ]:
grid_evt_['EVT_CLASS'].unique()

In [ ]:
df = grid_evt_.copy() # make a copy to work with
df = df[df['count'] > 0]
forest = ['Closed tree canopy', 'Open tree canopy', 'Sparse tree canopy']
df = df[df['EVT_CLASS'].isin(forest)]

col = 'SAF_SRM'

# Calculate the total percent cover (prevalence)
spp_prevalence_evt = df.groupby(col)['count'].sum().reset_index()
spp_prevalence_evt['fraction'] = spp_prevalence_evt['count'] / spp_prevalence_evt['count'].sum()
spp_prevalence_evt = spp_prevalence_evt.sort_values(by='fraction', ascending=False)

# which species make up 97% of observations?
spp_prevalence_evt['c_fraction'] = spp_prevalence_evt['fraction'].cumsum()
top_species_evt = spp_prevalence_evt[spp_prevalence_evt['c_fraction'] <= 0.97]
print(f"\n\nSpecies making up 97% of the total: \n\n{top_species_evt}\n\n")

# plot it
plt.figure(figsize=(8, 6))
sns.barplot(data=top_species_evt, x=col, y='fraction', palette='viridis')

plt.xlabel(f'Existing Vegetation Type ()')
plt.ylabel('Fractional cover')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# save the EVT summary table out.
out_fp = os.path.join(projdir, 'data/tabular/mod/viirs_snpp_jpss1_gridstats_lf-evt.csv')
grid_evt_.to_csv(out_fp)
print(f"Saved to: {out_fp}")

In [ ]:
# get aspen grids (any aspen component)
# get gridcells with at least some aspen
spp = 'SAF 217: Aspen'
spp_idx = grid_evt_[grid_evt_['SAF_SRM'] == spp]['grid_index'].unique() # list of grid indices
grid_spp = grid_evt_[grid_evt_['grid_index'].isin(spp_idx)]
# check how many aspen gridcells there are
n_spp_grids = len(grid_spp['grid_index'].unique())
print(f"There are a total of {n_spp_grids} [{round(n_spp_grids/len(grid_evt_['grid_index'].unique())*100,1)}%] grids w/ any aspen")

In [ ]:
# Identify the dominant EVT class for each observation
dominant_evt = grid_evt_.loc[grid_evt_.groupby('grid_index')['pct_cover'].idxmax()]
dominant_evt.head()

In [ ]:
n = len(dominant_evt[dominant_evt['SAF_SRM'] == 'SAF 217: Aspen'])
print(round(n/len(grid_evt_)*100,2))

In [ ]:
grid_evt_d = pd.merge(grid, dominant_evt, on='grid_index', how='inner')
grid_evt_d.columns

In [ ]:
# Plot FRP by EVT group name
# Find the median and sort the evt classes

col = 'SAF_SRM'

df = grid_evt_d.copy()
df = df[df['SAF_SRM'].isin(top_species_evt['SAF_SRM'].unique())]

med = df.groupby(col)['frp_max'].median().sort_values(ascending=False)
sorted = med.index.tolist()

df['frp_max_adj'] = df['frp_max'] + 1
df['frp_csum_adj'] = df['frp_csum'] + 1

# create a dataframe with dominance (>90% of the same species)
dom = df[df['pct_cover'] > 75]

plt.figure(figsize=(8, 6)) 

sns.boxplot(
    data=df, 
    x='frp_max_adj', 
    y=col, 
    order=sorted, 
    palette='coolwarm_r', 
    dodge=False)

plt.xscale('log')

plt.xlabel('Maximum Fire Radiative Power (FRP)')
plt.ylabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# Export a table of the EVT values found in AFD observations
lookup_ = lookup[lookup['VALUE'].isin(grid_evt_['evt'].unique())]
out_fp = os.path.join(projdir,'tabular/mod/EVT/afd_evt_codes-SRM.csv')
lookup_.to_csv(out_fp)
print(f"Saved to: {out_fp}")

In [ ]:
'Aspen Forest, Woodland, and Parkland',
'Douglas-fir-Ponderosa Pine-Lodgepole Pine Forest and Woodland',
'Spruce-Fir Forest and Woodland',
'Ponderosa Pine Forest, Woodland and Savanna',
'Big Sagebrush Shrubland and Steppe',
'Lodgepole Pine Forest and Woodland',
'Douglas-fir-Grand Fir-White Fir Forest and Woodland',
'Deciduous Shrubland', 
'Grassland',
'Aspen-Mixed Conifer Forest and Woodland',
'Pinyon-Juniper Woodland',
'Western Riparian Woodland and Shrubland',
'Mountain Mahogany Woodland and Shrubland',
'Douglas-fir Forest and Woodland',

## Sentinel-based Aspen Cover

In [ ]:
fp = os.path.join(maindir,'aspen-fire/Aim1/data/spatial/mod/results/classification/s2aspen_distribution_10m_y2019_CookEtAl.tif')
s2aspen_da = rxr.open_rasterio(fp, masked=True, cache=False, chunks='auto').squeeze()
shp, gt, wkt, nd = s2aspen_da.shape, s2aspen_da.spatial_ref.GeoTransform, s2aspen_da.rio.crs, s2aspen_da.rio.nodata
print(
    f"Shape: {shp}; \n"
    f"GeoTransform: {gt}; \n"
    f"WKT: {wkt}; \n"
    f"NoData Value: {nd}; \n"
    f"Data Type: {s2aspen_da[0].dtype}")
gc.collect()

In [ ]:
grid.dtypes

In [ ]:
# filter the grid to fires 2019-2023
grid['Fire_Year'] = grid['year'] = grid['dt_max_frp'].dt.year
grid_ = grid[grid['Fire_Year'] >= 2019]
print(f"There are {len(grid_['Fire_ID'].unique())} fires 2019-2023")

In [ ]:
t0 = time.time()

# Get the proportional landcover
grid_s2aspen = compute_band_stats(
    geoms=grid_[['grid_index','geometry']], 
    image_da=s2aspen_da, 
    id_col='grid_index',
    attr='s2aspen'
)

t1 = (time.time() - t0) / 60
print(f"Total elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

In [ ]:
grid_s2aspen = grid_s2aspen[grid_s2aspen['s2aspen'] == 1]
grid_s2aspen.head()

In [ ]:
round(len(grid_s2aspen)/len(grid_)*100,2)

In [ ]:
# get aspen grids (any aspen component)
# get gridcells with at least some aspen
spp_idx = grid_tm[grid_tm['species'] == spp]['grid_index'].unique() # list of grid indices
grid_spp = grid_tm[grid_tm['grid_index'].isin(spp_idx)]
# check how many aspen gridcells there are
n_spp_grids = len(grid_spp['grid_index'].unique())
print(f"There are a total of {n_spp_grids} [{round(n_spp_grids/len(grid_tm['grid_index'].unique())*100,1)}%] grids w/ any aspen")

In [ ]:
# combine aspen percent cover from TreeMap, EVT, and Sentinel

In [ ]:
# Identify aspen classes from the EVT data
aspen_classes = grid_evt_[grid_evt_['EVT_NAME'].str.contains('aspen', case=False, na=False)]['EVT_NAME'].unique()
print(f"Aspen EVT classes in the Southern Rockies:\n{aspen_classes}\n")
evt_aspen = grid_evt_[grid_evt_['EVT_NAME'].isin(aspen_classes)]
evt_aspen.head(3)

In [ ]:
# and from treemap
treemap_aspen = treemap_df[treemap_df['SpeciesName'] == 'Aspen']
treemap_aspen.head(3)

In [ ]:
# merge the dataframes on matching grid_index

# TreeMap
treemap_aspen_ = treemap_aspen[['grid_index','spp_pct','count','forest_pct']]
treemap_aspen_.rename(columns={'spp_pct': 'treemap_pct', 'count': 'treemap_count'}, inplace=True)
print(f"TreeMap: {len(treemap_aspen_)} aspen grids")

# landfire evt
evt_aspen_ = evt_aspen[['grid_index','EVT_NAME','pct_cover','count']]
evt_aspen_.rename(columns={'pct_cover': 'lfevt_pct', 'count': 'lfevt_count'}, inplace=True)
print(f"LANDFIRE EVT: {len(evt_aspen_)} aspen grids")

# sentinel
grid_s2aspen_ = grid_s2aspen[['grid_index','pct_cover','count']]
grid_s2aspen_.rename(columns={'pct_cover': 'sentinel_pct', 'count': 'sentinel_count'}, inplace=True)
print(f"Sentinel: {len(grid_s2aspen_)} aspen grids")

# combine them
aspen_df = treemap_aspen_.merge(evt_aspen_, on='grid_index', how='outer')
aspen_df = aspen_df.merge(grid_s2aspen_, on='grid_index', how='outer')
# aspen_df = aspen_df[['grid_index','treemap_pct','treemap_count',
#                      'lfevt_pct','lfevt_count','sentinel_pct','sentinel_count',
#                      'EVT_NAME','forest_pct']]
aspen_df = aspen_df.fillna(0) # set NaN to 0 (no aspen cover)
print(f"\n\tCombined: {len(aspen_df)} aspen grids.")
print("\n")
aspen_df.head(3)

In [ ]:
aspen_df_m = aspen_df.melt(
    id_vars=['grid_index', 'forest_pct'],
    value_vars=['treemap_pct', 'lfevt_pct', 'sentinel_pct'],
    var_name='source',
    value_name='pct_cover'
)

# Distribution plot
plt.figure(figsize=(6, 4))
sns.histplot(
    data=aspen_df_m,
    x='pct_cover',
    hue='source',
    kde=True,
    palette='viridis',
    binwidth=5,
    common_norm=False
)
plt.title('Distribution of Aspen Cover Percent by Data Source')
plt.xlabel('Aspen Cover Percent (%)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
aspen_df['evt_ha'] = aspen_df['lfevt_count'] * 900 / 10000  # EVT: 30m resolution
aspen_df['treemap_ha'] = aspen_df['treemap_count'] * 900 / 10000  # TreeMap: 30m resolution
aspen_df['sentinel_ha'] = aspen_df['sentinel_count'] * 100 / 10000  # Sentinel: 10m resolution

aspen_df[['grid_index', 'evt_ha', 'treemap_ha', 'sentinel_ha']].head()

In [ ]:
area_df = aspen_df.melt(
    id_vars=['grid_index'],
    value_vars=['evt_ha', 'treemap_ha', 'sentinel_ha'],
    var_name='source',
    value_name='area_ha'
)

# Plot the distributions
plt.figure(figsize=(6,3))
sns.histplot(
    data=area_df,
    x='area_ha',
    hue='source',
    kde=True,
    palette='viridis',
    binwidth=0.1,
    common_norm=False
)
plt.title('Distribution of Aspen Area (ha) by Data Source')
plt.xlabel('Aspen Area (ha)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# Summarize total aspen area for each source
total_areas = area_df.groupby('source')['area_ha'].sum().reset_index()

# Bar plot of total areas
plt.figure(figsize=(6, 3))
sns.barplot(data=total_areas, x='source', y='area_ha', palette='viridis')

plt.title('Total Aspen Area by Data Source')
plt.xlabel('Data Source')
plt.ylabel('Total Area (ha)')
plt.tight_layout()
plt.show()

In [ ]:
correlation_df = aspen_df[['evt_ha', 'treemap_ha', 'sentinel_ha']]
correlation_matrix = correlation_df.corr()

# Heatmap of correlations
plt.figure(figsize=(6, 4.75))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='viridis',
    cbar_kws={'label': 'Correlation Coefficient'}
)
plt.tight_layout()
plt.show()

In [ ]:
grid_aspen = pd.merge(grid, aspen_df, on='grid_index', how='inner')
grid_aspen.columns

In [ ]:
grid_aspen['treemap_dom'] = grid_aspen['treemap_pct'] > 50
grid_aspen['lfevt_dom'] = grid_aspen['lfevt_pct'] > 50
grid_aspen['sentinel_dom'] = grid_aspen['sentinel_pct'] > 50

# Count the number of dominant grids for each data source
dom_counts = grid_aspen[['treemap_dom', 'lfevt_dom', 'sentinel_dom']].sum()

print("Number of Aspen-Dominant Grids (>50%):")
print(dom_counts)

In [ ]:
aspen_dominant = grid_aspen.melt(
    id_vars=['grid_index', 'frp_max', 'frp_csum'],
    value_vars=['treemap_dom', 'lfevt_dom', 'sentinel_dom'],
    var_name='source',
    value_name='dominant'
)
print(aspen_dominant.head(3))

# Filter for dominant grids
aspen_dominant = aspen_dominant[aspen_dominant['dominant']]
aspen_dominant['frp_max_adj'] = aspen_dominant['frp_max'] + 1
aspen_dominant['frp_csum_adj'] = aspen_dominant['frp_csum'] + 1

# dom = aspen_dominant[aspen_dominant['pct_cover'] > 75]

# Plot boxplot
plt.figure(figsize=(6, 4))

sns.boxplot(
    data=aspen_dominant, 
    x='frp_max_adj', 
    y='source', 
    palette='coolwarm_r', 
    dodge=False)

plt.xscale('log')

plt.xlabel('Maximum Fire Radiative Power (FRP)')

plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import kruskal

# Prepare data for statistical testing
treemap_frp = grid_aspen.loc[grid_aspen['treemap_dom'], 'frp_max']
lfevt_frp = grid_aspen.loc[grid_aspen['lfevt_dom'], 'frp_max']
sentinel_frp = grid_aspen.loc[grid_aspen['sentinel_dom'], 'frp_max']

# Perform Kruskal-Wallis test
stat, p = kruskal(treemap_frp, lfevt_frp, sentinel_frp)
print(f"Kruskal-Wallis Test: H-statistic={stat:.3f}, p-value={p:.3g}")

if p < 0.05:
    print("Significant differences in FRP distributions between data sources.")
else:
    print("No significant differences in FRP distributions between data sources.")

In [ ]:
# Make some plots. Just for the SRM.

In [ ]:
# Join back to FRP
frp = afds[['afdID','INSTRUMENT','ACQ_DATETIME','DAYNIGHT','FRP','na_l3name']]
frp = frp[frp['na_l3name'] == 'Southern Rockies']

evt = afds_evt[['afdID','EVT_NAME','EVT_PHYS','EVT_GP_N','pct_cover']]

frp_evt_srm = pd.merge(frp, evt, on='afdID', how='left')
frp_evt_srm.head()

In [ ]:
# Create a "pct_forest" class
forest_classes = ['Hardwood', 'Conifer', 'Conifer-Hardwood']
frp_plots_forest = frp_evt_srm[frp_evt_srm['EVT_NAME'].isin(forest_classes)]
# Pivot table and calculate the percent forest
forest_cover = (
    frp_evt_srm[['afdID', 'EVT_PHYS', 'pct_cover']]
    .pivot_table(index='afdID', columns='EVT_PHYS', values='pct_cover', aggfunc='sum', fill_value=0)
)
forest_cover['pct_forest'] = forest_cover[forest_classes].sum(axis=1)
# Merge back to the main data frame
frp_evt_srm = frp_evt_srm.merge(forest_cover[['pct_forest']], on='afdID')
frp_evt_srm['pct_forest'].describe()

In [ ]:
# Now filter to retain just the aspen composition in FRP observations
frp_evt_srm_aspen = frp_evt_srm[frp_evt_srm['EVT_NAME'].isin(aspen_classes)]
# What percent of FRP observations have an aspen component?
aspenpct = len(frp_evt_srm_aspen['afdID'].unique()) / len(frp_evt_srm['afdID'].unique()) * 100
print(f"In the Southern Rockies, {aspenpct:.2f} of MODIS/VIIRS AFD have an aspen component.")

In [ ]:
plt.figure(figsize=(6, 4))

frp_plots_aspen_d = frp_evt_srm_aspen[frp_evt_srm_aspen['DAYNIGHT'] == 'D']
frp_plots_aspen_n = frp_evt_srm_aspen[frp_evt_srm_aspen['DAYNIGHT'] == 'N']

# Apply log transformation to FRP values; add 1 to avoid taking log of zero
sns.kdeplot(np.log1p(frp_plots_aspen_d['FRP']), fill=True, label="Daytime Obs.")
sns.kdeplot(np.log1p(frp_plots_aspen_n['FRP']), fill=True, label="Nighttime Obs.")

# Set plot title and axis labels
plt.title('Log density of FRP for any aspen (Southern Rockies)')
plt.xlabel('Log of Fire Radiative Power (FRP)')
plt.ylabel('Density')
plt.legend()
plt.tight_layout()  # Adjust layout to prevent clipping of ylabel

# plt.savefig(os.path.join(maindir,'aspen-fire/Aim2/figures/FigX_frp_density_aspen-any_day-night.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
plt.figure(figsize=(12, 4))

frp_plots_aspen_ = frp_evt_srm_aspen[frp_evt_srm_aspen['FRP'] > 0]

for aspen_class in aspen_classes:
    subset = frp_plots_aspen_[frp_plots_aspen_['EVT_NAME'] == aspen_class]
    sns.scatterplot(x='pct_cover', y='FRP', data=subset, label=aspen_class)

plt.yscale('log')

plt.title('FRP vs Percent Cover of Aspen Classes')
plt.xlabel('Percent Cover of Aspen')
plt.ylabel('Fire Radiative Power (FRP)')
plt.legend(title='Aspen Class')

# plt.savefig(os.path.join(maindir,'aspen-fire/Aim2/figures/FigX_aspen_frp_scatterplot.png'), dpi=300, bbox_inches='tight')

plt.show()

del frp_plots_aspen_

In [ ]:
# Identify the dominant EVT types.

In [ ]:
# Identify the dominant EVT class for each observation
dominant_evt = afds_evt.loc[afds_evt.groupby('afdID')['pct_cover'].idxmax()]
dominant_evt.head()

In [ ]:
print(f"There are {len(dominant_evt['EVT_GP_N'].unique())} EVT_GP_N categories.\n")
print(dominant_evt['EVT_GP_N'].unique())

In [ ]:
# Save this file out as well.
out_fp = os.path.join(projdir,'data/tabular/mod/EVT/afd_aspen-fires_2018_to_2023_buffer-EVT-dominant.csv')
dominant_evt.to_csv(out_fp)
print(f"Saved zonal statistics to: {out_fp}")

In [ ]:
# Join back to FRP
frp = afds[['afdID','INSTRUMENT','ACQ_DATETIME','DAYNIGHT','FRP','na_l3name']]
frp = frp[frp['na_l3name'] == 'Southern Rockies']
evt = dominant_evt[['afdID','EVT_NAME','EVT_PHYS','EVT_GP_N','pct_cover']]
frp_evt = pd.merge(frp, evt, on='afdID', how='left')
frp_evt.head()

In [ ]:
# Plot FRP by EVT group name

df = frp_evt[frp_evt['INSTRUMENT'] == 'VIIRS']

med_frp = df.groupby('EVT_GP_N')['FRP'].median().sort_values(ascending=False)
sorted = med_frp.index.tolist()

plt.figure(figsize=(10, 10)) 

sns.boxplot(
    data=df,
    x='FRP', 
    y='EVT_GP_N', 
    hue='DAYNIGHT',
    order=sorted,  
    palette='coolwarm', 
    dodge=True
)

plt.xscale('log')

plt.xlabel('Fire Radiative Power (FRP)')
plt.ylabel('Dominant Vegetation Type')

plt.tight_layout()
plt.show()

### Calculate the CBD, CBH, and CC from LANDFIRE

Now we calculate zonal statistics for these continuous attributes from LANDFIRE (e.g., mean within the AFD buffer zone).

In [ ]:
start = time.time()    

# Read in the LANDFIRE layers
cbd_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CBD_200_CONUS/Tif/LC20_CBD_200.tif')
cbh_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CBH_200_CONUS/Tif/LC20_CBH_200.tif')
tcc_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CC_200_CONUS/Tif/LC20_CC_200.tif')

lf_paths = [cbd_path, cbh_path, tcc_path]
attrs = ['cbd','cbh','tcc']

results = []
for i in range(len(lf_paths)):
    lf = lf_paths[i]
    attr = attrs[i]
    print(f'Processing {os.path.basename(lf)}')

    # Open the raster data and crop to bounds as before
    lf_da = rxr.open_rasterio(lf, masked=True, cache=False).squeeze()
    lf_da_crop = lf_da.rio.clip_box(
        minx=bounds[0]+1000, # +1000 meters for a small buffer
        miny=bounds[1]+1000, 
        maxx=bounds[2]+1000, 
        maxy=bounds[3]+1000
    )

    if not lf_da_crop.rio.crs == afds.crs:
        print("CRS mismatch, fixing !")
        lf_da_crop = lf_da_crop.rio.reproject(afds.crs)

    del lf_da # clean up
    
    # Calculate the zonal statistics
    zsdf = compute_band_stats(
        geoms=afds, 
        image_da=lf_da_crop, 
        id_col='afdID',
        attr=attr,
        stats=['mean'],
        ztype='continuous'
    )
    
    results.append(zsdf)
    del zsdf

print(f"\nTotal elapsed time: {round((time.time() - start)/60)} minutes")

In [ ]:
from functools import reduce
afd_lf_ = reduce(lambda left, right: pd.merge(left, right, on='afdID', how='inner'), results)
afd_lf_.head()

In [ ]:
# Join to the percent cover data
